In [1]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')
# plotting
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model

import urllib2 
import csv
import numpy as np
from sklearn import svm 
from sklearn.metrics import *
import matplotlib.pyplot as plt
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure import *
from pybrain.datasets import *
from pybrain.structure.modules import *
#! pip install pybrain

In [2]:
import urllib2 
import csv
import numpy as np
from sklearn import svm 
from sklearn.metrics import *
import sysdef readdef read_data(path):

In [3]:
def read_data(path):
    all_features = []
    time_list = []
    high_list = []
    low_list = []
    open_price_list =[]
    close_list = []
    total_volume_list = []
    volume_list = []
    count=0
    path = '/home/octo/Dropbox/IQ_data/'
    csv_file = open(path + 'SPY.csv', 'r')
    content = csv_file.readlines()
    for line in content:
        l=line.split(',')
        all_features.append(l)
        time,high, low, open_,close,total_volume,volume = l
        time_list.append(time)
        close_list.append(float(close))
        high_list.append(float(high))
        low_list.append(float(low))
        open_price_list.append(float(open_))
        volume_list.append(float(volume))
    return time_list, close_list, high_list, low_list, open_price_list, volume_list

In [4]:
def creating_binary_labels(close_list, open_price_list):
    label_list = close_list - open_price_list
    label_list = label_list[1:-1]

    for i in range(len(label_list)):
        if(label_list[i]>0):
            label_list[i]=1
        else:
            label_list[i]=0
    return label_list

In [5]:
def fearure_creation(timestamp_list, close_list, high_list, low_list, open_price_list, volume_list, x):
    #Initialising
    open_change_percentage_list=[]
    close_change_percentage_list=[]
    low_change_percentage_list=[]
    high_change_percentage_list=[]
    volume_change_percentage_list=[]    
    volume_diff_percentage_list=[]
    open_diff_percentage_list=[]
    Open_price_moving_average_list=[]
    Close_price_moving_average_list=[]
    High_price_moving_average_list=[]
    Low_price_moving_average_list=[]


    highest_open_price = open_price_list[0]
    lowest_open_price = open_price_list[0]
    highest_volume = volume_list[0]
    lowest_volume = volume_list[0]
    if(x>len(open_price_list)):
        x = len(open_price_list)
    for i in range(len(close_list)-x,len(close_list)):
        if(highest_open_price<open_price_list[i]):
            highest_open_price=open_price_list[i]
        if(lowest_open_price>open_price_list[i]):
            lowest_open_price=open_price_list[i]
        if(highest_volume<volume_list[i]):
            highest_volume=volume_list[i]
        if(lowest_volume>volume_list[i]):
            lowest_volume=volume_list[i]
    #Finding change percentage list/difference list
    opensum=open_price_list[0]
    closesum=close_list[0]
    highsum=high_list[0]
    lowsum=low_list[0]
    for i in range(1, len(close_list)-1):
        close_change_percentage = (close_list[i] - close_list[i-1])/close_list[i-1]
        close_change_percentage_list.append(close_change_percentage)
        
        open_change_percentage = (open_price_list[i+1] - open_price_list[i])/open_price_list[i]
        open_change_percentage_list.append(open_change_percentage)

        high_change_percentage = (high_list[i] - high_list[i-1])/high_list[i-1]
        high_change_percentage_list.append(high_change_percentage)
        if volume_list[i-1]==0:
            volume_list[i-1] = volume_list[i-2]

        volume_change_percentage = (volume_list[i] - volume_list[i-1])/volume_list[i-1]
        volume_change_percentage_list.append(volume_change_percentage)

        low_change_percentage = (low_list[i] - low_list[i-1])/low_list[i-1]
        low_change_percentage_list.append(low_change_percentage)


        volume_diff = (volume_list[i] - volume_list[i-1])/(highest_volume-lowest_volume)
        volume_diff_percentage_list.append( volume_diff)

        open_diff = (open_price_list[i+1] - open_price_list[i])/(highest_open_price - lowest_open_price)
        open_diff_percentage_list.append(open_diff)

        opensum+=open_price_list[i]
        closesum+=close_list[i]
        highsum+=high_list[i]
        lowsum+=low_list[i]

        Open_price_moving_average = float(opensum/i+1) / open_price_list[i+1]
        Open_price_moving_average_list.append(Open_price_moving_average)

        High_price_moving_average = float(highsum/i+1) / high_list[i+1]
        High_price_moving_average_list.append(High_price_moving_average)

        Close_price_moving_average = float(closesum/i+1) / close_list[i+1]
        Close_price_moving_average_list.append(Close_price_moving_average)

        Low_price_moving_average = float(lowsum/i+1) / low_list[i+1]
        Low_price_moving_average_list.append(Low_price_moving_average)
    #Combining features
    close_change_percentage_list = np.array(close_change_percentage_list)
    high_change_percentage_list = np.array(high_change_percentage_list)
    low_change_percentage_list = np.array(low_change_percentage_list)
    volume_change_percentage_list = np.array(volume_change_percentage_list)
    open_price_list = np.array(open_price_list)
    close_list = np.array(close_list)
    open_diff_percentage_list=np.array(open_diff_percentage_list)
    volume_change_percentage_list=np.array(volume_change_percentage_list)
    
    feature1 = np.column_stack((open_change_percentage_list, close_change_percentage_list, high_change_percentage_list, low_change_percentage_list, volume_change_percentage_list))  
    feature2 = np.column_stack((open_change_percentage_list, close_change_percentage_list, high_change_percentage_list, low_change_percentage_list, volume_change_percentage_list, open_diff_percentage_list, volume_diff_percentage_list))
    
    feature3 = np.column_stack((open_change_percentage_list, close_change_percentage_list, high_change_percentage_list, low_change_percentage_list, volume_change_percentage_list, open_diff_percentage_list, volume_diff_percentage_list,Open_price_moving_average_list, Close_price_moving_average_list, High_price_moving_average_list, Low_price_moving_average_list))
    
    label_list = creating_binary_labels(close_list, open_price_list)
    return feature1, feature2, feature3, label_list

In [6]:
def svm_rbf(feature, label_list): 
    length_feature = len(feature)
    index = 5
    predicted_list=[]
    test_label_list=[]
    accuracy_score_list=[]
    for i in range(length_feature-5):
        train_feature = feature[0:index]
        test_feature = feature[index:index+1]
        train_label = label_list[0:index]
        test_label = label_list[index:index+1]
        clf = svm.SVC(C=100000,kernel='rbf')
        clf.fit(train_feature, train_label)
        predicted = clf.predict(test_feature)
        predicted_list.append(predicted[0])
        test_label_list.append(test_label)
        index+=1
        Accuracy = accuracy_score(predicted_list, test_label_list)        
        #print "Accuracy: ", Accuracy
        accuracy_score_list.append(Accuracy)
     

    Accuracy = accuracy_score(predicted_list, test_label_list)       
    print "Final Accuracy: ", Accuracy
    Precision = precision_score(predicted_list, test_label_list)
    Recall = recall_score(predicted_list, test_label_list)
    print "Precision Score :", Precision
    print "Recall Score :" , Recall
    return accuracy_score_list

In [7]:
def plotting_accuracy(accuracy, name):
    plt.plot(accuracy)
    plt.savefig(name)
    plt.close()

In [8]:
def  plotting_svm(predicted, test_labels,name,clr):
    step = np.arange(0, len(test_labels))
    plt.subplot(211)
    plt.xlim(-1, len(test_labels) + 1)
    plt.ylim(-1, 2)
    plt.ylabel('Actual Values')
    plt.plot(step, test_labels, drawstyle = 'step',color=clr)
    plt.subplot(212)
    plt.xlim(-1, len(test_labels) + 1)
    plt.ylim(-1, 2)
    plt.xlabel('minutes')
    plt.ylabel('Predicted Values')
    plt.plot(step, predicted, drawstyle = 'step',color=clr)
    plt.savefig(name)
    plt.close()

In [9]:
def neural_networks(train_feature, train_label, test_features, test_labels):
    net = buildNetwork(len(train_feature[0]), 30, 1, hiddenclass = TanhLayer, outclass = TanhLayer,recurrent = True)
    ds = ClassificationDataSet(len(train_feature[0]), 1)
    for i, j in zip(train_feature, train_label):
        ds.addSample(i, j)
    trainer = BackpropTrainer(net, ds)
    epochs = 13
    for i in range(epochs):
        trainer.train()
    predicted = list()
    for i in test_features:
        predicted.append(int(net.activate(i)>0.5))
    predicted = np.array(predicted)

    print "Accuracy:", accuracy_score(test_labels, predicted)*100, "%"
    return predicted

In [ ]:
timestamp_list, close_list, high_list, low_list, open_price_list, volume_list = read_data('/home/octo/Dropbox/IQ_data/SPY.csv')
x = 5
feature1, feature2, feature3, label_list = fearure_creation(timestamp_list, close_list, high_list, low_list, open_price_list, volume_list, x )
    
print
print "-----------------------------------------------------------------------"
print "SVM - RBF Kernel with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%"
predicted1, test_label1, train_feature1, train_label1, test_feature1 = svm_rbf(feature1, label_list)
print "-----------------------------------------------------------------------"


-----------------------------------------------------------------------
SVM - RBF Kernel with Features : 
Open Change%, Close Change%, High Change%, Low Change%, Volume Change%


In [ ]:
print
print "-----------------------------------------------------------------------"
print "SVM - RBF Kernel with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%,"
print "Open Difference% , Volume Difference%, "
predicted2, test_label2, train_feature2, train_label2, test_feature2= svm_rbf(feature2, label_list)
print "-----------------------------------------------------------------------"
print
print "-----------------------------------------------------------------------"
print "SVM - RBF Kernel with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%,"
print "Open MovingAvg, Close MovingAvg, High MovingAvg, Low MovingAvg"
predicted3, test_label3, train_feature3, train_label3, test_feature3= svm_rbf(feature3, label_list)
print "-----------------------------------------------------------------------"
   
print "-----------------------------------------------------------------------"
print "SVM - RBF Kernel with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%"
print "Open Difference% , Volume Difference%, Open Price Moving Avg"
print "Close Price Moving Avg, High Price Moving Avg, Low Price Moving Avg"
predicted4, test_label4, train_feature4, train_label4, test_feature4 =  svm_rbf(feature4, label_list)
print "-----------------------------------------------------------------------"
    
plotting_svm(predicted1, test_label1,"Plots/tcs/tcs1_22_apr_rbf.jpg",'r')
plotting_svm(predicted2, test_label2,"Plots/tcs/tcs2_22_apr_rbf.jpg",'g')
plotting_svm(predicted3, test_label3,"Plots/tcs/tcs3_22_apr_rbf.jpg",'b')
plotting_svm(predicted4, test_label4,"Plots/tcs/tcs4_22_apr_rbf.jpg",'y')

In [ ]:
print
print "*******************************RNN*************************************"
print
print "-----------------------------------------------------------------------"
#print "RNN with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%"
predicted_NN_1=neural_networks(train_feature1, train_label1, test_feature1, test_label1)
print "-----------------------------------------------------------------------"

print
print "-----------------------------------------------------------------------"
#print "RNN with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%,"
print "Open Difference% , Volume Difference%, "
predicted_NN_2=neural_networks(train_feature2, train_label2, test_feature2, test_label2)
print "-----------------------------------------------------------------------"
   
print
print "-----------------------------------------------------------------------"
#print "RNN with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%,"
print "Open MovingAvg, Close MovingAvg, High MovingAvg, Low MovingAvg"
predicted_NN_3=neural_networks(train_feature3, train_label3, test_feature3, test_label3)
print "-----------------------------------------------------------------------"

print
print "-----------------------------------------------------------------------"
#print "RNN with Features : "
print "Open Change%, Close Change%, High Change%, Low Change%, Volume Change%"
print "Open Difference% , Volume Difference%, Open Price Moving Avg"
print "Close Price Moving Avg, High Price Moving Avg, Low Price Moving Avg"
predicted_NN_4=neural_networks(train_feature4, train_label4, test_feature4, test_label4)
print "-----------------------------------------------------------------------"
   
plotting_svm(predicted_NN_1, test_label1,"Plots/tcs/tcs1_22_apr_NN.jpg",'r')
plotting_svm(predicted_NN_2, test_label2,"Plots/tcs/tcs2_22_apr_NN.jpg",'g')
plotting_svm(predicted_NN_2, test_label3,"Plots/tcs/tcs3_22_apr_NN.jpg",'b')
plotting_svm(predicted_NN_2, test_label4,"Plots/tcs/tcs4_22_apr_NN.jpg",'y')   